# Noteboook destinada al entrenamiento del modelo

## 1. Importar librerías y raw data
Todas las librerías (Pandas, NumPy, Seaborn...) y el propio código modular `src/`.

In [20]:
import sys
sys.path.insert(0, '../src/')
import pandas as pd
import importlib
import feature_engineering 
importlib.reload(feature_engineering)

from feature_engineering import get_features, get_cat_num_features


In [35]:
# Imporar dataset preprocesado
df = pd.read_csv('../data/processed/df_clean.csv')
target = 'minutos_visitados_mes'

### 2. Feature Engineering

- Creación de nuevas features

In [36]:
df_new = get_features(df)
df_new.head()

,cliente_id,aniomes,fecha,canal,region,minutos_visitados_mes,cantidad_heladeras,venta_total_negocios_mes,cantidad_productos_total_negocios_vendidos_mes,cantidad_compras_total_negocios_mes,...,venta_negocio1_mes_negativa,venta_negocio2_mes_negativa,venta_negocio3_mes_negativa,venta_negocio4_mes_negativa,ventas_por_visita,productos_por_compra,venta_promedio_x_compra,mes_sin,mes_cos,is_first_month
0,3e1f078ba07f177421f0aefa1884347d,202501,2025-01-01,AUTOSERVICIO,LITORAL,31.600000,5.0,5.00946,23.0,45.0,...,False,False,False,False,0.158527,0.511111,0.111321,5.000000e-01,0.866025,0
1,58ae3b6ad98d3e3e4ba2b404c9142e0f,202504,2025-04-01,AUTOSERVICIO,GBA MINORISTAS,214.683333,4.0,6.65658,11.0,19.0,...,False,False,False,False,0.031007,0.578947,0.350346,8.660254e-01,-0.500000,0
2,f3a854fb42adecbc313785f1f1629ffe,202407,2024-07-01,KIOSCOS/MAXIKIOSCOS,NEA,17.433333,0.0,0.00000,0.0,0.0,...,False,False,False,False,0.000000,0.000000,0.000000,-5.000000e-01,-0.866025,0
3,e7d278a2f3c239afeaf9526d7e275401,202412,2024-12-01,KIOSCOS/MAXIKIOSCOS,LITORAL,24.733333,2.0,0.00000,0.0,0.0,...,False,False,False,False,0.000000,0.000000,0.000000,-2.449294e-16,1.000000,0
4,3f4bd1f117b18afa131c48de15486652,202411,2024-11-01,KIOSCOS/MAXIKIOSCOS,GBA MINORISTAS,11.166667,2.0,0.00000,0.0,0.0,...,False,False,False,False,0.000000,0.000000,0.000000,-5.000000e-01,0.866025,0


- Normalización de variables numéricas

In [23]:
cat_features, num_features = get_cat_num_features(df_new)

3 variables categóricas
17 variables numéricas
20 variables en total


In [38]:
cat_features = [f for f in cat_features if f not in ['cliente_id']]
num_features = [f for f in num_features if f not in [target, 'aniomes']]

### Clasificación de clientes

### 3.  Modelo Baseline